# 11 Defocus Blur

* 초점에 따라 장면의 일부분이 블러가 되는 효과를 구현합니다.

In [18]:
from typing import List
import sys
from tqdm import tqdm

import numpy as np
import numpy.linalg as LA

import matplotlib.pyplot as plt

infinity = sys.float_info.max

In [19]:
def normalize(vec: np.array, eps: float = 1e-6):
    return vec / (LA.norm(vec) + eps)


def blend(color1: np.array, color2: np.array, t: float):
    return (1.0 - t) * color1 + t * color2


def linear_to_gamma(linear_component: np.array) -> np.array:
    return np.where(linear_component > 0, np.sqrt(linear_component), linear_component)


def scale_color(color: np.array, num_sample_per_pixel: int) -> np.array:
    # divide the color by the number of samples.
    scale = 1.0 / num_sample_per_pixel
    rgb = scale * color
    rgb = np.clip(rgb, 0.0, 0.999)
    return rgb


def random() -> float:
    return np.random.uniform(low=0.0, high=1.0)


def random_in_unit_sphere() -> np.array:
    p = np.random.uniform(low=-1.0, high=1.0, size=[3])
    return normalize(p)


def random_in_unit_disk() -> np.array:    
    while True:
        p = np.array([np.random.uniform(-1,1), np.random.uniform(-1,1), 0.0])
        if np.dot(p, p) < 1.0:
            return p


def random_in_hemisphere(normal: np.array) -> np.array:
    p = random_in_unit_sphere()
    return p if np.dot(p, normal) > 0.0 else -p


def near_zero(vec: np.array, eps: float = 1e-6) -> bool:
    return np.all(np.abs(vec) < eps)


def reflect(vec: np.array, normal: np.array) -> np.array:
    return vec - 2 * np.dot(vec, normal) * normal


def refract(vec: np.array, normal: np.array, etai_over_etat: float) -> np.array:
    cos_theta = np.minimum(np.dot(-vec, normal), 1.0)
    r_out_perp = etai_over_etat * (vec + cos_theta * normal)
    r_out_parallel = -np.sqrt(np.abs(1.0 - np.dot(r_out_perp, r_out_perp))) * normal
    return r_out_perp + r_out_parallel

In [20]:
class Ray:
    def __init__(self, origin: np.array, direction: np.array) -> None:
        self.origin: np.array = origin
        self.direction: np.array = normalize(direction)

    def at(self, t: float) -> np.array:
        return self.origin + t * self.direction

In [21]:
class Material:
    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array, np.array]:
        return False, None, None

In [22]:
class Lambertian(Material):
    def __init__(self, albedo: np.array):
        self.albedo = albedo

    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array, np.array]:
        scatter_direction = record.normal + random_in_unit_sphere()

        if near_zero(scatter_direction):
            scatter_direction = record.normal

        scattered = Ray(origin=record.point, direction=scatter_direction)
        return True, self.albedo, scattered

In [23]:
class Metal(Material):
    def __init__(self, albedo: np.array, fuzz: float):
        self.albedo = albedo
        self.fuzz = fuzz if fuzz < 1 else 1

    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array]:
        reflected = reflect(r_in.direction, record.normal)
        reflected = normalize(reflected) + (self.fuzz * random_in_unit_sphere())

        scattered = Ray(origin=record.point, direction=reflected)
        if np.dot(scattered.direction, record.normal) > 0:
            return True, self.albedo, scattered
        else:
            return False, self.albedo, scattered

In [24]:
class Dielectric(Material):
    def __init__(self, index_of_refraction: float):
        # Refractive index in vacuum or air, or the ratio of the material's refractive index over
        # the refractive index of the enclosing media
        self.index_of_refraction = index_of_refraction

    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array, np.array]:
        attenuation = np.array([1.0, 1.0, 1.0])
        refraction_ratio = 1.0 / self.index_of_refraction if record.is_front_face else self.index_of_refraction

        unit_direction = normalize(r_in.direction)
        refracted = refract(unit_direction, record.normal, refraction_ratio)

        scattered = Ray(origin=record.point, direction=refracted)
        return True, attenuation, scattered

In [25]:
class HitRecord:
    def __init__(self, point: np.array, normal: np.array, t: float) -> None:
        self.point = point
        self.normal = normal
        self.material: Material = None
        self.t = t
        self.is_front_face = True

    def set_face_normal(self, ray: Ray, outward_normal: np.array) -> None:
        self.is_front_face = np.dot(ray.direction, outward_normal) < 0
        self.normal = outward_normal if self.is_front_face else -outward_normal

In [26]:
class Hittable:
    def hit(self, ray: Ray, t_min: float, t_max: float) -> HitRecord:
        raise NotImplementedError

In [27]:
class Sphere(Hittable):
    def __init__(self, center: np.array, radius: float, material: Material) -> None:
        self.center = center
        self.radius = radius
        self.material = material

    def hit(self, ray: Ray, t_min: float, t_max: float) -> bool:
        dir_center_to_origin = ray.origin - self.center

        a = np.dot(ray.direction, ray.direction)
        half_b = np.dot(dir_center_to_origin, ray.direction)
        c = np.dot(dir_center_to_origin, dir_center_to_origin) - self.radius**2.0

        discriminant = half_b**2 - a * c
        if discriminant < 0:
            return None
        sqrt_d = np.sqrt(discriminant)

        # find the nearest root that lies in the acceptable range.
        t = (-half_b - sqrt_d) / a
        if t < t_min or t_max < t:
            t = (-half_b + sqrt_d) / a
            if t < t_min or t_max < t:
                return None

        point = ray.at(t)
        outward_normal = (point - self.center) / self.radius
        hit_record = HitRecord(point=point, normal=outward_normal, t=t)
        hit_record.set_face_normal(ray=ray, outward_normal=outward_normal)
        hit_record.material = self.material

        return hit_record

In [28]:
class HittableList(Hittable):
    def __init__(self) -> None:
        self.objects: List[Hittable] = []

    def clear(self) -> None:
        self.objects.clear()

    def add(self, object: Hittable) -> None:
        self.objects.append(object)

    def hit(self, ray: Ray, t_min: float, t_max: float) -> HitRecord:
        record = None
        closest_so_far = t_max

        for object in self.objects:
            tmp_record = object.hit(ray=ray, t_min=t_min, t_max=closest_so_far)
            if tmp_record is not None:
                closest_so_far = tmp_record.t
                record = tmp_record

        return record

In [29]:
def ray_color(ray: Ray, world: Hittable, depth: int) -> np.array:
    if depth <= 0:
        color = np.zeros(3)
        return color

    record = world.hit(ray=ray, t_min=0.0, t_max=infinity)
    if record is not None:
        scatter_info = record.material.scatter(ray, record)
        if scatter_info[0]:
            return scatter_info[1] * ray_color(ray=scatter_info[2], world=world, depth=depth - 1)
        return np.zeros(3)

    t = 0.5 * (ray.direction[1] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    return blend(color1=color1, color2=color2, t=t)

In [30]:
class Camera:
    def __init__(
            self,
            aspect_ratio: float = 1.0, 
            image_width: int = 100,             
            num_sample_per_pixel: int = 50,
            max_depth: int = 5,
            vfov: float = 90.0,
            lookfrom: np.array = np.array([0.0, 0.0, 0.0]),
            lookat: np.array = np.array([0.0, 0.0, -1.0]),
            vup: np.array = np.array([0.0, 1.0, 0.0]),            
            defocus_angle: float = 0,
            focus_dist: float = 10.0,
        ) -> None:

        self.aspect_ratio = aspect_ratio
        self.image_width = image_width        
        self.vfov = vfov
        self.num_sample_per_pixel = num_sample_per_pixel
        self.max_depth = max_depth
        self.lookfrom = lookfrom
        self.lookat = lookat
        self.vup = vup
        self.defocus_angle = defocus_angle
        self.focus_dist = focus_dist

    def _prepare(self):
        self.image_height = int(self.image_width / self.aspect_ratio)
        self.image_height = 1 if (self.image_height < 1) else self.image_height

        self.center = self.lookfrom
        
        theta = np.radians(self.vfov)
        h = np.tan(theta / 2.0)
        viewport_height = 2.0 * h * self.focus_dist
        viewport_width = self.aspect_ratio * viewport_height

        w = normalize(self.lookfrom - self.lookat)
        u = normalize(np.cross(self.vup, w))
        v = np.cross(w, u)

        self.horizontal_vec = viewport_width * u
        self.vertical_vec = viewport_height * v
        self.frontal_vec = self.focus_dist * w
        self.lower_left_corner = (
            self.center
            - self.horizontal_vec / 2.0
            - self.vertical_vec / 2.0
            - self.frontal_vec
        )

        self.pixel_delta_u = self.horizontal_vec / self.image_width
        self.pixel_delta_v = self.vertical_vec / self.image_height

        # Calculate the camera defocus disk basis vectors.
        defocus_radius = self.focus_dist * np.tan(np.radians(self.defocus_angle / 2.0))
        self.defocus_disk_u = u * defocus_radius
        self.defocus_disk_v = v * defocus_radius

    def _defocus_disk_sample(self) -> np.array:
        p = random_in_unit_disk()
        return self.center + (p[0] * self.defocus_disk_u) + (p[1] * self.defocus_disk_v)
        
    def _get_ray(self, u: float, v: float) -> Ray:
        pixel_sample = self.lower_left_corner + u * self.pixel_delta_u + v * self.pixel_delta_v
        ray_origin = self.center if (self.defocus_angle <= 0) else self._defocus_disk_sample()
        return Ray(origin=ray_origin, direction=pixel_sample - ray_origin)
    
    def render(self, world: HittableList) -> np.ndarray:
        self._prepare()
        
        image = np.zeros([self.image_height, self.image_width, 3])

        for row in tqdm(range(self.image_height)):
            for col in range(self.image_width):
                for _ in range(self.num_sample_per_pixel):
                    u = col + np.random.uniform(low=0.0, high=1.0)
                    v = row + np.random.uniform(low=0.0, high=1.0)
                    ray = self._get_ray(u, v)
                    image[row, col] += ray_color(ray=ray, world=world, depth=self.max_depth)

        image = scale_color(image, num_sample_per_pixel=self.num_sample_per_pixel)
        image = linear_to_gamma(image)
        return image        

In [31]:
# material
material_ground = Lambertian(np.array([0.8, 0.8, 0.0]))
material_center = Lambertian(np.array([0.1, 0.2, 0.5]))
material_left = Dielectric(1.5)
material_bubble = Dielectric(1.0 / 1.5)
material_right = Metal(np.array([0.8, 0.6, 0.2]), 1.0)

# world
world = HittableList()
world.add(Sphere(center=np.array([0.0, -100.5, -1.0]), radius=100.0, material=material_ground))
world.add(Sphere(center=np.array([0.0, 0.0, -1.2]), radius=0.5, material=material_center))
world.add(Sphere(center=np.array([-1.0, 0.0, -1.0]), radius=0.5, material=material_left))
world.add(Sphere(center=np.array([-1.0, 0.0, -1.0]), radius=0.4, material=material_bubble))
world.add(Sphere(center=np.array([ 1.0, 0.0, -1.0]), radius=0.5, material=material_right))

In [32]:
# camera
camera = Camera(
    aspect_ratio=16.0/9.0, 
    image_width=100, 
    num_sample_per_pixel=50,
    max_depth=5,
    vfov=20.0,  # vertical view angle (field of view)    
    lookfrom = np.array([-2, 2, 1]),
    lookat = np.array([0, 0, -1]),
    vup = np.array([0, 1, 0]),    
    defocus_angle=10.0,
    focus_dist=3.4,
)

In [ ]:
image = camera.render(world)

plt.imshow(image, origin="lower")
plt.show()